# Operaciones básicas en Spark
- Spark opera con colecciones **inmutables y distribuidas** de elementos, manipulándolos en paralelo
    - API estructurada: DataFrames y DataSets
    - API de bajo nivel: RDDs (ya obsoleto ya que el API estructurada es más eficiente y más de alto nivel)

-   Operaciones sobre estas colecciones
    -   Creación
    -   Transformaciones (ordenación, filtrado, etc.)
    -   Realización acciones para obtener resultados

-   Spark automáticamente distribuye los datos y paraleliza las operaciones

### Ejemplo: creación de un DataFrame a partir de un fichero CSV
En este ejemplo, Spark infiere el esquema de los datos de forma automática

  - Es preferible especificar el esquema de forma explícita, como veremos más adelante

También se especifica que la primera línea es la cabecera.

### Dataset de ejemplo: Vuelos internacionales 2015

El archivo `2015-summary.csv` contiene un resumen de vuelos internacionales en 2015:
- **DEST_COUNTRY_NAME**: País de destino
- **ORIGIN_COUNTRY_NAME**: País de origen
- **count**: Número de vuelos entre ese par de países

Es un dataset pequeño (~256 filas) ideal para aprender Spark sin sobrecargar recursos.

In [ ]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/dsevilla/tcdm-public/24-25/datos/2015-summary.csv",
    "2015-summary.csv",
)
!ls -lh 2015-summary.csv
!head 2015-summary.csv

In [ ]:
%pip install pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Creamos un objeto SparkSession (o lo obtenemos si ya está creado)
spark: SparkSession = (
    SparkSession.builder.appName("Mi aplicacion")
    # Reducir particiones de shuffle para datasets pequeños (default: 200)
    # (sólo se pone como ejemplo de cómo cambiar parámetros)
    .config("spark.sql.shuffle.partitions", "4")
    .master("local[*]")  # Usar todos los cores disponibles localmente
    .getOrCreate()
)

# SparkContext solo es necesario para trabajar con RDDs (API antigua, no recomendada)
# Con DataFrames/SQL normalmente no lo necesitas
sc: SparkContext = spark.sparkContext

In [ ]:
from pyspark.sql.dataframe import DataFrame

datosVuelos2015: DataFrame = (
    spark.read.option("inferSchema", "true").option("header", "true").csv("2015-summary.csv")
)

In [ ]:
datosVuelos2015.printSchema()

In [ ]:
datosVuelos2015.show()
print(datosVuelos2015.count())

assert datosVuelos2015.count() == 256

In [ ]:
datosVuelos2015.show(5)

### Rows

Las filas de un DataFrame son objetos de tipo `Row`

- API de Row en Python: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Row.html

In [ ]:
# Obtenemos las dos primeras fila del DataFrame
from pyspark.sql.types import Row

rows1_2: list[Row] = datosVuelos2015.take(2)
print(rows1_2)
print(type(rows1_2))
print(type(rows1_2[0]))

In [ ]:
# Obtén la primera fila como un diccionario Python
print(rows1_2[0].asDict())
print(type(rows1_2[0].asDict()))

assert type(rows1_2[0].asDict()) is dict

**¿Cuándo es útil `Row.asDict()`?**
- **Integración con código Python puro**: convertir datos Spark a estructuras nativas
- **Serialización a JSON**: para APIs REST o almacenamiento
- **Debugging**: inspeccionar valores específicos de forma legible
- **Procesamiento individual**: cuando necesitas trabajar con una fila concreta fuera de Spark

### Particiones

Spark divide las filas DataFrame en un conjunto de particiones

-   El número de particiones por defecto es función del tamaño del cluster (número total de cores en todos los ejecutores) y del tamaño de los datos (número de bloques de los ficheros en HDFS)
-   Para RDDs se puede especificar otro valor en el momento de crearlos
-   También se puede modificar una vez creados

**¿Por qué importan las particiones?**
- **Demasiadas particiones** → overhead de coordinación entre tareas
- **Muy pocas particiones** → no se aprovecha el paralelismo disponible
- **Regla general**: 2-4 particiones por CPU core disponible en el cluster
- **Para shuffle operations**: `spark.sql.shuffle.partitions` (default: 200, puede ser excesivo para datos pequeños)

In [ ]:
print(f"Número de particiones: {datosVuelos2015.rdd.getNumPartitions()}")

# Creo un nuevo DataFrame con 4 particiones
datosVuelos2015_4P: DataFrame = datosVuelos2015.repartition(4)
print(f"Número de particiones: {datosVuelos2015_4P.rdd.getNumPartitions()}")

assert datosVuelos2015_4P.rdd.getNumPartitions() == 4

### Transformaciones

Operaciones que transforman los datos

  - No modifican los datos de origen (*inmutabilidad*)
  - Se computan de forma "perezosa" (*lazyness*) - ver sección siguiente

Dos tipos:

  - **Transformaciones estrechas (narrow)**
    - Cada partición de entrada contribuye a una única partición de salida
    - No se modifica el número de particiones
    - Normalmente se realizan en memoria
    - **Ejemplos**: `select()`, `filter()`, `withColumn()`, `map()`
    
  - **Transformaciones anchas (wide)**
    - Cada partición de salida depende de varias (o todas) particiones de entrada
    - Suponen un barajado de datos (shuffle)
    - Pueden implicar un cambio en el número de particiones
    - Pueden suponer escrituras en disco
    - **Ejemplos**: `groupBy()`, `sort()`/`orderBy()`, `join()`, `distinct()`

In [ ]:
# Ejemplo de una transformación narrow: reemplazar valores
# replace() modifica valores en todas las columnas de tipo string
datosVuelos2015_EEUU: DataFrame = datosVuelos2015.replace("United States", "Estados Unidos")

# Verificar el cambio (narrow: solo filtra filas)
print("Antes del replace:")
datosVuelos2015.filter(datosVuelos2015.DEST_COUNTRY_NAME == "United States").show(3)

print("\nDespués del replace:")
datosVuelos2015_EEUU.filter(datosVuelos2015_EEUU.DEST_COUNTRY_NAME == "Estados Unidos").show(3)

In [ ]:
# Ejemplo de una transformación wide: ordenar
# sort() requiere shuffle porque todas las particiones necesitan compararse
datosVuelos2015_Ord: DataFrame = datosVuelos2015_EEUU.sort("count", ascending=False)

# cache() mantiene el DataFrame en memoria para reutilizarlo sin recalcular
# Útil cuando vas a usar el mismo DataFrame varias veces (ej: en acciones múltiples)
datosVuelos2015_Ord.cache()

print("Top 5 rutas con más vuelos:")
datosVuelos2015_Ord.show(5)

### Concepto clave: Lazy Evaluation (Evaluación Perezosa)

**Las transformaciones NO se ejecutan inmediatamente**

Cuando escribes:
```python
df_filtrado = df.filter(col("edad") > 18)
df_ordenado = df_filtrado.orderBy("nombre")
```

Spark **NO ejecuta** estas operaciones todavía. En su lugar:

1. **Construye un plan de ejecución** (DAG - Grafo Dirigido Acíclico)
2. **Optimiza el plan** (elimina pasos innecesarios, reordena operaciones)
3. **Solo ejecuta** cuando llamas una **acción** (ej: `.show()`, `.count()`, `.collect()`)

**Ventajas:**
- Spark puede optimizar todo el pipeline antes de ejecutar
- Evita cálculos innecesarios si solo necesitas una muestra
- Permite fusionar transformaciones para mayor eficiencia

**Ejemplo**: Si haces `.filter().map().filter()`, Spark puede combinar los dos filtros en uno solo

### Acciones

Obtienen un resultado, forzando a que se realicen las transformaciones pendientes

  - En el momento de disparar la *acción* se crea un *plan* con las transformaciones necesarias para obtener los datos solicitados
    - Se crea un Grafo Dirigido Acíclico (DAG) conectando las transformaciones
    - Spark optimiza ese grafo, para eliminar transformaciones innecesarias o unir las que sea posible
  - Las acciones traducen el DAG en un plan de ejecución

Tipos de acciones

  - Acciones para mostrar datos por consola
  - Acciones para convertir datos Spark en datos del lenguaje
  - Acciones para escribir datos a disco


In [ ]:
# Ejemplo de acciones
from pprint import pp

print(f"Número de filas en la tabla: {datosVuelos2015_Ord.count()}")

pp(datosVuelos2015_Ord.take(3))

datosVuelos2015_Ord.show()

### Visualizando el plan de ejecución

Spark permite inspeccionar el plan físico que ejecutará

In [ ]:
# Ver el plan de ejecución físico optimizado
print("Plan de ejecución para datosVuelos2015_Ord:")
datosVuelos2015_Ord.explain(mode="formatted")

# También puedes usar mode="simple", "extended", o "cost" para más detalles

**Spark UI**: Para una visualización más rica del DAG y métricas de ejecución:
- Accede a `http://localhost:4040` mientras tu aplicación Spark está ejecutándose
- Muestra: stages, tasks, storage, environment, SQL queries con planes visuales

### Ejemplo completo: Pipeline típico en Spark

Veamos un ejemplo que integra todo lo aprendido: transformaciones narrow y wide, acciones, y optimización

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

# Pipeline completo: análisis de países con más tráfico aéreo
resultado: DataFrame = (
    # 1. Leer datos con esquema inferido
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv("2015-summary.csv")
    # 2. Filtrar solo rutas con tráfico significativo (transformación narrow)
    .filter(col("count") > 100)
    # 3. Agrupar por país de destino (transformación wide - shuffle)
    .groupBy("DEST_COUNTRY_NAME")
    # 4. Calcular total de vuelos por destino (agregación)
    .agg(spark_sum("count").alias("total_vuelos"))
    # 5. Ordenar por total descendente (transformación wide - shuffle)
    .orderBy(col("total_vuelos").desc())
    # 6. Limitar a top 10 (transformación narrow)
    .limit(10)
)

# NOTA: Hasta aquí NO se ha ejecutado nada (lazy evaluation)
# Solo al llamar .show() se dispara la ejecución completa

print("Top 10 países destino por volumen de vuelos (rutas con >100 vuelos):\n")
resultado.show()  # Acción que dispara todo el pipeline

# Verificar que obtuvimos exactamente 10 resultados
assert resultado.count() == 10

**Análisis del pipeline anterior:**

1. **Transformaciones narrow**: `filter()`, `limit()` - no requieren shuffle
2. **Transformaciones wide**: `groupBy()` + `agg()`, `orderBy()` - requieren shuffle
3. **Optimización automática**: Spark reordena operaciones para minimizar datos procesados
4. **Lazy evaluation**: Todo el plan se ejecuta solo al llamar `.show()`

**Tip**: Usar `.explain()` en el resultado muestra cómo Spark optimizó el pipeline